In [1]:
# Importing libraries
import keras
import torch
import warnings

import numpy as np

warnings.simplefilter('ignore')

In [8]:
# Loading data
cifar10 = keras.datasets.cifar10.load_data()

imdb_reviews = keras.datasets.imdb.load_data(
    skip_top=50,
    num_words=5000,
    start_char=1,
    oov_char=2,
    index_from=3
)
word_index   = keras.datasets.imdb.get_word_index()
index_word   = {v + 3: k for k, v in word_index.items()}

index_word[0] = '<PAD>'
index_word[1] = '<START>'
index_word[2] = '<OOV>'

# CIFAR-10 Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

The classes are:

| Label |	Description |
|-------|-------------|
| 0     |	airplane    |
| 1     |	automobile  |
| 2     |	bird        |
| 3     |	cat         |
| 4     |	deer        |
| 5     |	dog         |
| 6     |	frog        |
| 7     |	horse       |
| 8     |	ship        |
| 9     |	truck       |

Source: https://www.cs.toronto.edu/~kriz/cifar.html

In [60]:
(x_train, y_train), (x_test, y_test) = cifar10
x_train = x_train / 255
x_test  = x_test / 255

print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_test shape: {y_test.shape}')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


## Keras MLP Image Classification

In [61]:
# Building model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 32, 3)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compiling model
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
    ]
)

# Fitting model
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=5,
    validation_data=(x_test, y_test)
)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 1.9575 - sparse_categorical_accuracy: 0.2943 - val_loss: 1.6978 - val_sparse_categorical_accuracy: 0.3964
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - loss: 1.6886 - sparse_categorical_accuracy: 0.4005 - val_loss: 1.6063 - val_sparse_categorical_accuracy: 0.4310
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - loss: 1.5980 - sparse_categorical_accuracy: 0.4351 - val_loss: 1.5611 - val_sparse_categorical_accuracy: 0.4492
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - loss: 1.5365 - sparse_categorical_accuracy: 0.4589 - val_loss: 1.5306 - val_sparse_categorical_accuracy: 0.4508
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - loss: 1.4844 - sparse_categorical_accuracy: 0.4759 - val_loss: 1.6258 - val_sparse_categorical_accuracy: 0.4203


## Pytorch MLP Image Classification

In [71]:
(x_train, y_train), (x_test, y_test) = cifar10

x_train = torch.from_numpy(x_train).float() / 255
x_test  = torch.from_numpy(x_test).float() / 255

y_train = torch.nn.functional.one_hot(torch.from_numpy(y_train).long(), num_classes=10).squeeze(1).float()
y_test  = torch.nn.functional.one_hot(torch.from_numpy(y_test).long(), num_classes=10).squeeze(1).float()

In [73]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(32*32*3, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 10),
    torch.nn.Softmax(dim=1),
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    train_loss = 0
    train_acc  = 0
    for batch in range(len(x_train) // 32):
        x_batch = x_train[batch * 32: (batch + 1) * 32]
        y_batch = y_train[batch * 32: (batch + 1) * 32]

        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == y_batch.argmax(dim=1)).sum().item()

    train_loss /= (batch + 1)
    train_acc  /= len(x_train)

    model.eval()
    with torch.no_grad():
      test_loss = 0
      test_acc  = 0
      for batch in range(len(x_test) // 32):
          x_batch = x_test[batch * 32: (batch + 1) * 32]
          y_batch = y_test[batch * 32: (batch + 1) * 32]

          y_pred = model(x_batch)
          loss = loss_fn(y_pred, y_batch)

          test_loss += loss.item()
          test_acc  += (y_pred.argmax(dim=1) == y_batch.argmax(dim=1)).sum().item()

      test_loss /= (batch + 1)
      test_acc  /= len(x_test)

    print(f'Epoch: {epoch + 1}')
    print(f'Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}')
    print(f'Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}')
    print("-" * 50)

Epoch: 1
Train loss: 2.2946 | Train acc: 0.1115
Test loss: 2.2831 | Test acc: 0.1259
--------------------------------------------------
Epoch: 2
Train loss: 2.2628 | Train acc: 0.1642
Test loss: 2.2396 | Test acc: 0.1974
--------------------------------------------------
Epoch: 3
Train loss: 2.2165 | Train acc: 0.2446
Test loss: 2.1958 | Test acc: 0.2645
--------------------------------------------------
Epoch: 4
Train loss: 2.1857 | Train acc: 0.2727
Test loss: 2.1766 | Test acc: 0.2764
--------------------------------------------------
Epoch: 5
Train loss: 2.1714 | Train acc: 0.2854
Test loss: 2.1653 | Test acc: 0.2885
--------------------------------------------------


## IMDB Movie reviews

This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset.

Source: https://ai.stanford.edu/~amaas/data/sentiment/

## Keras MLP Text Classification

In [23]:
(x_train, y_train), (x_test, y_test) = imdb_reviews

x_train = keras.utils.pad_sequences(x_train, maxlen=500)
x_test  = keras.utils.pad_sequences(x_test,  maxlen=500)

In [24]:
# Building model
model = keras.Sequential([
    keras.layers.Embedding(5000, 32),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

# Compiling model
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
    ]
)

# Fitting model
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=5,
    validation_data=(x_test, y_test)
)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 79s 99ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5071 - val_loss: 0.6925 - val_sparse_categorical_accuracy: 0.5154
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 84ms/step - loss: 0.6916 - sparse_categorical_accuracy: 0.5228 - val_loss: 0.6923 - val_sparse_categorical_accuracy: 0.5169
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 85ms/step - loss: 0.6911 - sparse_categorical_accuracy: 0.5227 - val_loss: 0.6920 - val_sparse_categorical_accuracy: 0.5157
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - loss: 0.6898 - sparse_categorical_accuracy: 0.5324 - val_loss: 0.6919 - val_sparse_categorical_accuracy: 0.5155
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 85ms/step - loss: 0.6882 - sparse_categorical_accuracy: 0.5416 - val_loss: 0.6914 - val_sparse_categorical_accuracy: 0.5200


## Pytorch MLP Text Classification

In [25]:
(x_train, y_train), (x_test, y_test) = imdb_reviews

x_train = torch.from_numpy(keras.utils.pad_sequences(x_train, maxlen=500))
x_test  = torch.from_numpy(keras.utils.pad_sequences(x_test,  maxlen=500))

y_train = torch.nn.functional.one_hot(torch.from_numpy(y_train).long(), num_classes=2).float()
y_test  = torch.nn.functional.one_hot(torch.from_numpy(y_test).long(), num_classes=2).float()

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Embedding(5000, 32),
    torch.nn.Flatten(),
    torch.nn.Linear(16000, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 2),
    torch.nn.Softmax(dim=1),
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    train_loss = 0
    train_acc  = 0
    for batch in range(len(x_train) // 32):
        x_batch = x_train[batch * 32: (batch + 1) * 32]
        y_batch = y_train[batch * 32: (batch + 1) * 32]

        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == y_batch.argmax(dim=1)).sum().item()

    train_loss /= (batch + 1)
    train_acc  /= len(x_train)

    model.eval()
    with torch.no_grad():
      test_loss = 0
      test_acc  = 0
      for batch in range(len(x_test) // 32):
          x_batch = x_test[batch * 32: (batch + 1) * 32]
          y_batch = y_test[batch * 32: (batch + 1) * 32]

          y_pred = model(x_batch)
          loss = loss_fn(y_pred, y_batch)

          test_loss += loss.item()
          test_acc  += (y_pred.argmax(dim=1) == y_batch.argmax(dim=1)).sum().item()

      test_loss /= (batch + 1)
      test_acc  /= len(x_test)

    print(f'Epoch: {epoch + 1}')
    print(f'Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}')
    print(f'Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}')
    print("-" * 50)